常见的脏数据种类有：  

数据值缺失  
数据值异常大或小  
格式错误  
非独立数据错误    
  
先自己自造一份脏数据

In [1]:
raw_data = [
["Name", "StudentID", "Age", "AttendClass", "Score"],
["小明", 20131, 10, 1, 67],
["小花", 20132, 11, 1, 88],
["小菜", 20133, None, 1, "98"],
["小七", 20134, 8, 1, 110],
["花菜", 20134, 98, 0, None],
["刘欣", 20136, 12, 0, 12]
]
print(raw_data)

[['Name', 'StudentID', 'Age', 'AttendClass', 'Score'], ['小明', 20131, 10, 1, 67], ['小花', 20132, 11, 1, 88], ['小菜', 20133, None, 1, '98'], ['小七', 20134, 8, 1, 110], ['花菜', 20134, 98, 0, None], ['刘欣', 20136, 12, 0, 12]]


首先，我们要做的就是把这份 Python list 数据转化成 Numpy 数据。其实用 Python 列表也能做数据清洗工作， 但为什么要用 Numpy 来做呢？这不得不提到 Numpy 面对大数据时，它得天独厚的计算速度优势了。同样一份 1GB 的数据，Python List 处理起来可能要 1 小时， Numpy 优化后，速度可以成倍增长。

In [2]:
import numpy as np
data = np.array(raw_data)
data

array([['Name', 'StudentID', 'Age', 'AttendClass', 'Score'],
       ['小明', 20131, 10, 1, 67],
       ['小花', 20132, 11, 1, 88],
       ['小菜', 20133, None, 1, '98'],
       ['小七', 20134, 8, 1, 110],
       ['花菜', 20134, 98, 0, None],
       ['刘欣', 20136, 12, 0, 12]], dtype=object)

结尾处有一个标识。dtype=object，这是什么意思呢？（剧透一下：这种 dtype 会对后续数据处理带来很多麻烦） 我们对比一下如果是纯数据的格式，numpy 的 dtype 是什么样。

In [3]:
test1 = np.array([1,2,3])
test2 = np.array([1.1,2.3,3.4])
test3 = np.array([1,2,3], dtype=np.float64)
print("test1.dtype", test1.dtype)
print("test2.dtype", test2.dtype)
print("test3.dtype", test3.dtype)

print("test2 > 2 ", test2 > 2)
print("data > 2", data > 2) # 这里会报错

test1.dtype int32
test2.dtype float64
test3.dtype float64
test2 > 2  [False  True  True]


TypeError: '>' not supported between instances of 'str' and 'int'

不同的 array 创建方式，可能会有不同的 dtype，而之前的 data.dtype=object，说明Python list 直接转换过来的 data 是无法参与诸多 Numpy 计算的。 而只有 dtype 为 int，float 这样的数值形式，才能参与运算。

# 数据预处理
我们筛选过滤一下。

In [4]:
data_process = []
for i in range(len(raw_data)):
    if i == 0:
        continue    # 不要首行字符串
    # 去掉首列名字
    data_process.append(raw_data[i][1:])
data = np.array(data_process, dtype=np.float64)
print("data.dtype", data.dtype)
print(data) 

data.dtype float64
[[2.0131e+04 1.0000e+01 1.0000e+00 6.7000e+01]
 [2.0132e+04 1.1000e+01 1.0000e+00 8.8000e+01]
 [2.0133e+04        nan 1.0000e+00 9.8000e+01]
 [2.0134e+04 8.0000e+00 1.0000e+00 1.1000e+02]
 [2.0134e+04 9.8000e+01 0.0000e+00        nan]
 [2.0136e+04 1.2000e+01 0.0000e+00 1.2000e+01]]


因为这份数据中存在 None，而只有 np.float 能转换 None

# 清洗数据
怎么清洗脏数据，取决于你要拿这个数据做什么，很明显，我们想要看看有没有什么数据是不合逻辑的。比如我发现学号有重复，可能是在输入学生信息的时候手误输错了。

In [5]:
sid = data[:, 0]
unique, counts = np.unique(sid, return_counts=True)
print(counts)

[1 1 1 2 1]


 np.unique() 功能我可能没说过，但是看名字就知道，它是用来查找数据中独一无二的数据的，return_counts 让我能清楚看到是数据的重复数量。 比如有一个数据重复出现了 2 次。我们来看看是哪个数据出现了多次。

In [6]:
print(unique[counts > 1])

[20134.]


然后综合判断，我们的数据中少了一个 20135，可能就是把某个同学的学号输错了，我们将错误的同学修改过来。

In [7]:
data[4, 0] = 20135 
print(data)

[[2.0131e+04 1.0000e+01 1.0000e+00 6.7000e+01]
 [2.0132e+04 1.1000e+01 1.0000e+00 8.8000e+01]
 [2.0133e+04        nan 1.0000e+00 9.8000e+01]
 [2.0134e+04 8.0000e+00 1.0000e+00 1.1000e+02]
 [2.0135e+04 9.8000e+01 0.0000e+00        nan]
 [2.0136e+04 1.2000e+01 0.0000e+00 1.2000e+01]]


我们来看看第二列 Age 年龄。一眼可以发现，存在两个问题，有一个同学没有年龄。我们看看能不能用平均年龄补全这个缺失的信息。

In [8]:
is_nan = np.isnan(data[:,1])
print("is_nan:", is_nan)
nan_idx = np.argwhere(is_nan)

# 计算有数据的平均年龄，用 ~ 符号可以 True/False 对调
mean_age = data[~np.isnan(data[:,1]), 1].mean()
print("有数据的平均年龄：", mean_age)

is_nan: [False False  True False False False]
有数据的平均年龄： 27.8


为啥平均年龄有 27.8 岁，不正常啊！是的，在你处理自己的数据的时候，也会遇到不太正常的情况。   
我们居然有一个 98 岁的学生。 这不太正常啊，所以我觉得我也得把这个 98 岁的当异常数据看待。   
就用除开 98 和 nan 的数据， 计算其它数的平均值代替他们吧。

In [9]:
normal_age_mask = ~np.isnan(data[:,1]) & (data[:,1] < 20)
print("normal_age_mask:", normal_age_mask)

normal_age_mean = data[normal_age_mask, 1].mean()
print("normal_age_mean:", normal_age_mean)

data[~normal_age_mask, 1] = normal_age_mean
print("ages:", data[:, 1])

normal_age_mask: [ True  True False  True False  True]
normal_age_mean: 10.25
ages: [10.   11.   10.25  8.   10.25 12.  ]


到这一步之后，我们要看看上课和成绩的数据需不需要清洗。按理来说，如果没有上课（0），课程的成绩应该是不存在的（nan）；上课了（1）才会有成绩。 所以我们观察下面两组数据：

In [10]:
print(data[-3:, 2:])

[[  1. 110.]
 [  0.  nan]
 [  0.  12.]]


倒数第二行的数据还算是 ok 的，因为没上课，就没成绩，但是倒数第一行，没上课，怎么还有成绩？还有倒数第三行，成绩居然超出了满分 100 分。这些情况都是我们要处理的情况。

In [11]:
data[data[:,2] == 0, 3] = np.nan

# 超过 100 分和低于 0 分的都处理一下
data[:, 3] = np.clip(data[:, 3], 0, 100)

print(data[:, 2:])

[[  1.  67.]
 [  1.  88.]
 [  1.  98.]
 [  1. 100.]
 [  0.  nan]
 [  0.  nan]]


In [12]:
data

array([[2.0131e+04, 1.0000e+01, 1.0000e+00, 6.7000e+01],
       [2.0132e+04, 1.1000e+01, 1.0000e+00, 8.8000e+01],
       [2.0133e+04, 1.0250e+01, 1.0000e+00, 9.8000e+01],
       [2.0134e+04, 8.0000e+00, 1.0000e+00, 1.0000e+02],
       [2.0135e+04, 1.0250e+01, 0.0000e+00,        nan],
       [2.0136e+04, 1.2000e+01, 0.0000e+00,        nan]])